# Assignment 5 - Time Series Analysis
## Data Science Tools I
### Professor: Don Dalton

---

### Student: Duncan Ferguson

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Question 1 - Importing the Datasets

## Part (a)
### 8 points

Download the .csv files (CurrentPrices, EmploymentRate, ProductionVolume, and Sales) linked on Canvas on the assignment page and place them in a local repository alongside your notebook file. (If on Google Colab, upload the files to the colab.) Use Pandas to read each .csv file into its own variable.

After inspecting the first few rows of each data frame, clean up each data frame by converting the `Date` column to Datetime objects, then setting the index of the data frame to be the `Date` values to convert the data frame into a series.

Plot each dataset with an appropriate title, either individually in separate code cells or all together in a single plot using subplots.

In [ ]:
CurrentPrices = pd.read_csv('CurrentPrices.csv')
EmploymentRate = pd.read_csv('EmploymentRate.csv')
ProductionVolume = pd.read_csv('ProductionVolume.csv')
Sales = pd.read_csv('Sales.csv')


## Part (b)
### 4 points

Based on the plots you created in part (a), determine which pattern each dataset falls into and suggest a corresponding forecasting technique to match each pattern. For pattern and forecasting technique, limit your choice to the following options:

> **Patterns:** Horizontal, Trend, Seasonal without Trend, Seasonal with Trend

> **Forecasting Technique:** Naive Forecasting, Moving Average Smoothing, Exponential Smoothing, Regression Analysis, Regression Analysis with Dummy Variables

To answer this question, EDIT THIS TEXT CELL by replacing the x's below with your answers (as opposed to typing in a code cell).

Dataset | Pattern | Forecasting Technique
--- | --- | ---
CurrentPrice | x | x
EmploymentRate | x | x
ProductionVolume | x | x
Sales | x | x

# Question 2 - Examining the Data

For this question and the subsequent ones, we will use the **ProductionVolume** dataset as the time series of interest.

## Part (a)
### 4 points

To start, check the stationarity of the data by plotting the rolling mean and standard deviation alongside the data itself. 

Based on your plot, is the data stationary? Provide reasons with your answes. Include your answer either as a text cell or in-line comment.

## Part (b)
### 4 points

Use `seasonal_decompose` to observe the trend, seasonality, and residuals for the data by plotting all plots alongside one another.

Based on your plots, is there a cyclical pattern in the data? Provide reasons with your answers. Include your answer either as a text cell or in-line comment.

## Part (c)
### 4 points

To further confirm stationarity or non-stationarity, perform the Dickey-Fuller Test on the data. You may use the `run_adftest` method provided in lecture.

Based on the result, is any differencing needed? If so, first-order difference the data and re-run the Dickey-Fuller Test to see if the results change.

# Question 3 - Model Estimation

## Part (a)
### 4 points

Plot the auto-correlation function (ACF) to get an indication of what type of ARIMA model may apply to this dataset. Refer to the table below to determine which model is best.

Provide your model choice either in a text cell below or as comments in the code cell alongside your code for the ACF plot. 

Shape | Indicated Model
--- | ---
Exponential, decay to zero | AR - Use PACF plot to identify order
Alternating positive and negative, decaying to zero | AR - Use PACF plot to identify order
One or more spikes, rest are essentially zero  | MA - Order identified by where plot becomes zero
Decay, starting after a few lags | ARMA
All zero or close to zero | Data is essentially random
High values at fixed intervals | Include seasonal autoregressive term / difference
No decay to zero | The series is not stationary

## Part (b)
### 5 points

Plot the partial auto-correlation function (PACF). Based on the PACF plot and ACF plot, determine what values are likely best for `p` and `q`. 

Question 2 Part (c) should have determined the best value for `d`. Fill in your answer for `p`, `d`, and `q` in this text cell:

`p=` YOUR_ANSWER_WITH_REASON

`d=` YOUR_ANSWER (reason provided in Question 2 Part (c))

`q=` YOUR_ANSWER_WITH_REASON

# Question 4 - Model Fitting



## Part (a)
### 6 points

Given your determination of `p`, `d`, and `q`, fit an ARIMA model with those parameters using the entire time series as training data. Get the predicted values from the model (calling `predict` with `d` as the start value and including `typ='levels'` if `d>0`) and compute the mean squared error (MSE) between the predictions and the actual values.

In addition to printing the resulting MSE, plot the predictions alongside the actual values.

## Part (b)
### 4 points

To further assess the accuracy of the model, plot the residual errors of the model both as a **histogram** and as a **scatterplot**.

Based on these plots, are the residuals normally distributed? Are the mean and variance constant over time? Provide reasons with your answers. Include your answer either as a text cell or in-line comment.

## Part (c)
### 2 points

Since the estimated parameters are not guaranteed to provide the best results, try fitting an ARIMA model with new parameters (different value for at least one of `p`, `d`, and `q`). Based on MSE, is your new model better or worse than your original in part (a)?

# Question 5 - Forecasting

For this question, use whichever model from question 4 ended up being the better one based on MSE.

## Part (a)
### 2 points

Use your model to forecast the next value and print the result.

## Part (b)
### 2 points

Use `plot_predict` to plot the actual data, the model's predictions for the actual data, and the confidence interval and forecasts for the next 50 steps.